In [1]:
# self-generated similar design, PEERS semantic matrix
import pandas as pd
import numpy as np
import pickle

In [2]:
# 1638 words
with open("../wordpools/wasnorm_wordpool.txt") as f:
    items = f.readlines()
items = [i.replace("\n","") for i in items]
items = np.array(items)

In [3]:
simu_sess_num = 1000
nlist = 42
n = 6
wordpool = np.arange(1,1639)

In [4]:
df_study = pd.DataFrame()
df_test = pd.DataFrame()
rng = np.random.default_rng(seed=42)

for sess in range(simu_sess_num):

    all_words = rng.choice(wordpool, nlist*2*n, replace = False)

    for lst in range(nlist):

        tmp_df_study = pd.DataFrame()
        tmp_df_test = pd.DataFrame()
        
        # choose present words
        pres_words = all_words[lst*2*n:(lst+1)*2*n]
        pres_words = np.reshape(pres_words, (n,2))
        tmp_df_study['study_itemno1'] = pres_words[:,0]
        tmp_df_study['study_itemno2'] = pres_words[:,1]
        tmp_df_study['study_item1'] = items[pres_words[:,0]-1]
        tmp_df_study['study_item2'] = items[pres_words[:,1]-1]
        tmp_df_study['list'] = lst
        tmp_df_study['session'] = sess
        df_study = pd.concat([df_study,tmp_df_study])
        
        # choose test words
        order = np.arange(5,-1,-1)
        direction = rng.permutation([0] * 3 + [1] * 3)
        test_probe = pres_words[order,direction]
        correct_ans = pres_words[order,1-direction]
        tmp_df_test['test_itemno'] = test_probe
        tmp_df_test['test_item'] = items[test_probe-1]
        tmp_df_test['correct_ans'] = correct_ans
        tmp_df_test['lag'] = np.arange(0,12,2)
        tmp_df_test['list'] = lst
        tmp_df_test['session'] = sess
        tmp_df_test['order'] = direction + 1  # 1 = forward, 2 = backward
        df_test = pd.concat([df_test,tmp_df_test])

df_study.reset_index(inplace = True, drop=True)
df_test.reset_index(inplace = True, drop=True)  

In [5]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,list,session
0,1569,1107,VINEGAR,PORCUPINE,0,0
1,30,696,APARTMENT,HEADBAND,0,0
2,1457,1443,THIGH,TECHNICIAN,0,0
3,1331,31,SNACK,APE,0,0
4,1377,1579,STEP,WAIST,0,0
...,...,...,...,...,...,...
251995,842,1335,LOLLIPOP,SNOB,41,999
251996,260,57,CATHEDRAL,ATTIRE,41,999
251997,946,1306,NET,SIGNATURE,41,999
251998,164,981,BRACES,ORCHESTRA,41,999


In [6]:
df_test

,test_itemno,test_item,correct_ans,lag,list,session,order
0,468,DINNER,681,0,0,0,2
1,1579,WAIST,1377,2,0,0,2
2,1331,SNACK,31,4,0,0,1
3,1457,THIGH,1443,6,0,0,1
4,696,HEADBAND,30,8,0,0,2
...,...,...,...,...,...,...,...
251995,981,ORCHESTRA,164,2,41,999,2
251996,946,NET,1306,4,41,999,1
251997,57,ATTIRE,260,6,41,999,2
251998,842,LOLLIPOP,1335,8,41,999,1


In [7]:
# save
with open('simu6a_data/simu6a_design.pkl', 'wb') as outp:
    pickle.dump(df_study, outp, pickle.HIGHEST_PROTOCOL)
    pickle.dump(df_test, outp, pickle.HIGHEST_PROTOCOL)